# $\mathbf{\{0,1\}}$ - Integer Programming with Complete Constraint Matrices

## Random Informed Guesses

We begin by enumerating (given an integer $m$) all possible linear combinations on the columns of the complete matrix. Based on those, we try to filter the ones that are withing the ranges:
* $\mathbf{1} \cdot 2^{m-2} \leq \mathbf{b} \leq \mathbf{1} \cdot  2^{m-1}$
* $\mathbf{1} \cdot 2^{m-3} \leq \mathbf{b} \leq \mathbf{1} \cdot  2^{m-2}$
* $\mathbf{1} \cdot 2^{m-4} \leq \mathbf{b} \leq \mathbf{3} \cdot  2^{m-2}$
* $\vdots$

In [10]:
from helpers import *
from tqdm import tqdm
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [22]:
ranges_5 = {}
m = 5
for i in range(16):
    pickle_in = open("ComputationalResults/ranges"+str(5)+"/ranges"+str(m)+"_"+str(i)+".pkl", 'rb')
    d: dict = pickle.load(pickle_in)
    if i == 0:
        ranges_5 = d
    else:
        for key in d.keys():
            ranges_5[key] += d[key]

# with open("ComputationalResults/ranges"+str(5)+"_UBLB/ranges"+str(m)+"_UBLB.pkl", 'wb') as f:
#     pickle.dump(ranges_5_UBLB, f)
#     f.close()



| Range of Values               | Number of combinations |
|-------------------------------|------------------------|
| $2^{m-2} \leq b \leq 2^{m-1}$ | $546433766$            |
| $2^{m-3} \leq b \leq 2^{m-2}$ | $463546849$            |
| $2^{m-4} \leq b \leq 2^{m-3}$ | $1762382$              |
| $2^{m-5} \leq b \leq 2^{m-4}$ | $6995$                 |

| Range of Values               | Number of combinations |
|-------------------------------|------------------------|
| $2^{m-3} \leq b \leq 2^{m-1}$ | $2054665066$           |
| $2^{m-4} \leq b \leq 2^{m-2}$ | $543768948$            |
| $2^{m-5} \leq b \leq 2^{m-3}$ | $2178649$              |

| Range of Values               | Number of combinations |
|-------------------------------|------------------------|
| $2^{m-4} \leq b \leq 2^{m-1}$ | $2144799235$           |
| $2^{m-5} \leq b \leq 2^{m-2}$ | $546271135$            |

| Range of Values               | Number of combinations |
|-------------------------------|------------------------|
| $2^{m-5} \leq b \leq 2^{m-1}$ | $2147321017$           |

| Range of Values         | Number of combinations |
|-------------------------|------------------------|
| $0 \leq b \leq 2^{m-1}$ | $2147483648$           |
| $0 \leq b \leq 2^{m-2}$ | $546433766$            |
| $0 \leq b \leq 2^{m-3}$ | $2232875$              |
| $0 \leq b \leq 2^{m-4}$ | $9736$                 |
| $0 \leq b \leq 2^{m-5}$ | $203$                  |

In [29]:
import json
with open('ComputationalResults/feasRHS_4', 'rb') as f:
    feasRHS_4 = pickle.load(f)
    f.close()
feas = []
for rhs in feasRHS_4:
    rhs = json.loads(rhs)
    feas.append(rhs)
feas = np.array(feas)

KeyboardInterrupt: 

Fractionality depends on the determinant of the matrix because of Kramer's rule

## Claims & Intuitions

$\text{\bf Claim}$: If a right hand side $\mathbf{b} = (b_1, \dots, b_m)$ is feasible then $\max_i\{b_i\}-\min_i\{b_i\} \leq 2^{m-2}$

$\text{\bf Proof}$: Without loss of generality, assume that $\mathbf{b}$ is sorted. We claim, that if $\mathbf{b}$ is a feasible right hand side, then $b_1-b_m \leq 2^{m-2}$.

Suppose for the sake of contradiction that this is not the case. Then $b_1-b_m > 2^{m-2}$, and in particular we have that $b_1 > 2^{m-2}$.

Let $\mathbf{x}$ be a solution for $\mathbf{Ax} = \mathbf{b}$, and $C$ be the set of columns used to obtain that solution. Let $C_1 \subseteq C$ be the subset of columns with a leading one. It must hold that $|C_1|>2^{m-2}$. In addition, we can define $\mathbf{A}_1$ to be the submatrix of $\mathbf{A}$ composed of columns containing a $1$ as the first element.

We have that each row of $\mathbf{A}_1$ contains exactly $2^{m-2}$ ones, except the first one which contains $2^{m-1}$ ones. Additionally the elements of $C_1$ are columns of $\mathbf{A}_1$. Then since $|C_1|>2^{m-2}$, then each row of the sub-matrix of $\mathbf{A}_1$ formed by the columns in $C_1$ contains at least $b_1-2^{m-2}$ ones.

In particular this implies that $b_m \geq b_1-2^{m-2}$ and it follows that $b_1 - b_m \leq 2^{m-2}$. qed

$\textbf{Intuition 1}$: Let $\mathbf{b} \in \mathbb{N}^m$ be a vector and $\mathbf{b}_{\text{sort}$ be the sorted version of $\mathbf{b}$. Then $\mathbf{b}$ is feasible $\iff \mathbf{b}_{\text{sort}$ is feasible.